[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Piphi5/Adopt-a-Pixel3km-Notebooks/blob/main/AOI%20Generation/AOI%20Generation.ipynb)

# Specify Center Coordinates

Put your latitude and longitude here. It is recommended that you utilize Google Maps to find the coordinates of the desired starting point.

In [1]:
centerlat = 40.3428
centerlon = -105.6836
name = "Matteo Kimura"

# Generate Points

Running the cell will generate the AOI Grid points.

In [2]:
import os
import sys

import requests

if "google.colab" in sys.modules or (
    "GITHUB_JOB" in os.environ and os.environ["GITHUB_JOB"] == "colab-test"
):
    url = "https://raw.githubusercontent.com/Piphi5/Adopt-a-Pixel3km-Notebooks/main/utils/code/grid_constants.py"
    exec(requests.get(url).text)
else:
    module_path = os.path.abspath(os.path.join("../utils/code"))
    sys.path.append(module_path)
    from grid_constants import *

In [3]:
import folium
import pandas as pd

latitude_const, longitude_const = get_latlon_constants(500, centerlat)
# Create folium visualization map
m = folium.Map(location=[centerlat, centerlon], zoom_start=15)

# List that will store all grid points
mapper_coordinates = []

# Add center point
mapper_coordinates.append([centerlat, centerlon])

# Find the top left grid point by going 1250m N and 1250m W
top_left_lat = centerlat + 5 / 2 * latitude_const
top_left_lon = centerlon - 5 / 2 * longitude_const

# generates points and adds to list
for i in range(6):
    for j in range(6):
        latitude = top_left_lat - i * latitude_const
        longitude = top_left_lon + j * longitude_const
        mapper_coordinates.append([latitude, longitude])


for point in mapper_coordinates:
    folium.Marker(point).add_to(m)

# Visualize the points

After generating the points from the previous cell, run the following cell to put your AOI grid points onto an interactive map.

In [4]:
m

# Export to CSV

`directory`: (*For local/non-Colab notebooks*) The directory to save the file to. 

Exports to a CSV with the following naming convention:`{name}_AOI_Points_{center latitude}, {center longitude}_{timestamp (YYYY-MM-DD HH:MM:SS)}.csv`. The CSV should download directly to your computer.

Now you can upload this CSV file onto services like ArcGIS or Google MyMaps to help you get directions on navigating to these points.

In [5]:
directory = "Output Files"

In [6]:
from datetime import datetime

now = datetime.now()
# creates a csv file with Latitude and Longitude for use in Google My Maps or ArcGIS
mapper_df = pd.DataFrame(mapper_coordinates, columns=["Latitude", "Longitude"])
filename = f"{name}_AOI Points_{centerlat}, {centerlon}_{now:%Y-%m-%d %H-%M-%S}.csv"

if not os.path.exists(directory):
    os.mkdir(directory)
    mapper_df.to_csv(f"{directory}/{filename}")

if "google.colab" in str(get_ipython()):
    from google.colab import files

    files.download(filename)

# Google MyMaps example:

Go to [MyMaps](https://www.google.com/maps/d/)

Then, you can create a new map:
![Create new Map](https://github.com/Piphi5/Adopt-a-Pixel3km-Notebooks/raw/main/AOI%20Generation/Demo%20Photos/make-new-map.png)

Then, you can import the CSV File through this button:
![Import CSV](https://github.com/Piphi5/Adopt-a-Pixel3km-Notebooks/raw/main/AOI%20Generation/Demo%20Photos/select-import.png)

Then, specify the latitude and longitude columns:

![Specify Latlon Columns](https://github.com/Piphi5/Adopt-a-Pixel3km-Notebooks/raw/main/AOI%20Generation/Demo%20Photos/setup-columns.png)

Specify the naming columns:

![Specify naming Columns](https://github.com/Piphi5/Adopt-a-Pixel3km-Notebooks/raw/main/AOI%20Generation/Demo%20Photos/setup-title.png)

You can now access these items on google maps on your mobile device:

First, select the Saved Button:
![Select Saved](https://github.com/Piphi5/Adopt-a-Pixel3km-Notebooks/raw/main/AOI%20Generation/Demo%20Photos/GoogleMapsSelectSaved.png)

Once there, you can view the Maps button:
![Select Maps](https://drive.google.com/uc?export=view&id=1Yy1ccqa8fIKU5YhVlrjwFSauayWUyVMs)

Once there you can select your mymaps map:
![Select Map](https://drive.google.com/uc?export=view&id=1UTUIAF224QYz9XZ10lnCg0tnl5LGayP7)

Then Press on any point on the map to get directions for it:

![Get Direction](https://drive.google.com/uc?export=view&id=1foVP0V6JyObjUzvLxklq5s3CJs8blbWK)

## Theoretical Basis

In order to generate equally spaced out grid points with a certain distance, we need to figure out constants $\\delta \\varphi$ and $\\delta \\lambda$ that can be added onto the latitude and longitude coordinates, respectively, to specify a coordinate a certain distance (e.g. 500 meters) away.

### Main formula

Utilizing the [Haversine Formula](https://en.wikipedia.org/wiki/Haversine_formula), the distance between two points on a sphere is:

$d = 2r \\arcsin{\\sqrt{\\sin{(\\frac{\\varphi_2 - \\varphi_1}{2})}^2 + \\cos{\\varphi_1} \\cos{\\varphi_2} \\sin{(\\frac{\\lambda_2 -\\lambda_1}{2})}^2}}$

Where $\\varphi_1, \\varphi_2$ are the latitudes of the two points in radians, $\\lambda_1, \\lambda_2$ are the longitudes of the two points in radians, $r$ is the radius of the earth, and $d$ is the distance between the two points.

### Latitude Constant

To translate d meters in the North/South latitude direction, we can assume that $\\lambda_1 = \\lambda_2 \\Rightarrow \\lambda_1 - \\lambda_2 = 0$, therefore:

$d = 2r \\arcsin{\\sqrt{\\sin{(\\frac{\\varphi_2 - \\varphi_1}{2})^2}}}$

$\\delta \\varphi = \\varphi_2 - \\varphi_1 = 2 \\arcsin{(\\sin{(\\frac{d}{2r})})} : (rad) =  \\frac{360}{\\pi} \\arcsin{(\\sin{(\\frac{d}{2r})})} : (degrees)$

### Longitude Constant

To translate d meters in the East/West longitude direction, we can assume that $\\varphi = \\varphi_1 = \\varphi_2 \\Rightarrow \\varphi_1 - \\varphi_2 = 0$, therefore:

$d = 2r \\arcsin{\\sqrt{\\cos{\\varphi_1}\\cos{\\varphi_2} \\sin{(\\frac{\\lambda_2 - \\lambda_1}{2})^2}}}$

$\\delta \\lambda = \\lambda_2 - \\lambda_1 = 2 \\arcsin{(\\frac{\\sin{(\\frac{d}{2r})}}{\\cos{\\varphi}})} : (rad)$

$ = \\frac{360}{\\pi} \\arcsin{(\\frac{\\sin{(\\frac{d}{2r})}}{\\cos{\\varphi}})}: (degrees)$